In [1]:
!pip install -q torch scikit-learn
!pip install -q --upgrade pymatgen mp-api matminer
!pip install -q joblib matplotlib tqdm
print("✅ Installation complete!")

✅ Installation complete!


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib
from pymatgen.core.composition import Composition
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital
import matplotlib.pyplot as plt
from tqdm import tqdm
from mp_api.client import MPRester
import time
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

### Configuration

In [3]:
MP_API_KEY = 'Your API KEY'

if MP_API_KEY != 'Your API KEY' or not MP_API_KEY or len(MP_API_KEY)<10:
    raise ValueError("❌ Add your Materials Project API key from https://materialsproject.org/api")

TARGET_SAMPLES = 30000
EPOCHS = 200
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


print("MATERIALS PROPERTY PREDICTION")
print(f"Device: {DEVICE}")



MATERIALS PROPERTY PREDICTION
Device: cuda


### property cofiguration

In [4]:
PROPERTIES = {
    "band_gap": {
        "field": "band_gap",
        "units": "eV",
        "description": "Band Gap",
        "architecture": "resnet",
        "resnet_blocks": 4
    },
    "formation_energy": {
        "field": "formation_energy_per_atom",
        "units": "eV/atom",
        "description": "Formation Energy",
        "architecture": "vanilla"
    },
    "energy_above_hull": {
        "field": "energy_above_hull",
        "units": "eV/atom",
        "description": "Energy Above Hull",
        "architecture": "vanilla"
    },
    "density": {
        "field": "density",
        "units": "g/cm³",
        "description": "Density",
        "architecture": "vanilla"
    },
    "volume_per_atom": {
        "field": "volume",
        "units": "ų/atom",
        "description": "Volume per Atom",
        "architecture": "vanilla"
    },
    "total_magnetization": {
        "field": "total_magnetization",
        "units": "µB",
        "description": "Magnetization",
        "architecture": "resnet",
        "resnet_blocks": 4
    },
    "bulk_modulus": {
        "field": "bulk_modulus",
        "units": "GPa",
        "description": "Bulk Modulus",
        "architecture": "vanilla"
    },
    "shear_modulus": {
        "field": "shear_modulus",
        "units": "GPa",
        "description": "Shear Modulus",
        "architecture": "vanilla"
    },
    "elastic_anisotropy": {
        "field": "universal_anisotropy",
        "units": "",
        "description": "Elastic Anisotropy",
        "architecture": "vanilla"
    },
    "poissons_ratio": {
        "field": "homogeneous_poisson",
        "units": "",
        "description": "Poisson's Ratio",
        "architecture": "vanilla"
    },
    "dielectric_total": {
        "field": "total",
        "units": "",
        "description": "Dielectric (Total)",
        "architecture": "resnet",
        "resnet_blocks": 6
    },
    "dielectric_electronic": {
        "field": "electronic",
        "units": "",
        "description": "Dielectric (Electronic)",
        "architecture": "resnet",
        "resnet_blocks": 6
    },
    "dielectric_ionic": {
        "field": "ionic",
        "units": "",
        "description": "Dielectric (Ionic)",
        "architecture": "vanilla"
    },
    "piezoelectric_max": {
        "field": "e_ij_max",
        "units": "C/m²",
        "description": "Piezoelectric Max",
        "architecture": "vanilla"
    },
}

### Neural network Architecture

In [5]:
class ResidualBlock(nn.Module):
    """Residual block with skip connections"""
    def __init__(self, hidden_size, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x

        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.bn2(out)

        out = out + identity  # Skip connection
        out = self.relu(out)

        return out

class ResNetPredictor(nn.Module):
    """ResNet for complex materials properties"""
    def __init__(self, input_size, hidden_size=256, num_blocks=4, dropout=0.3):
        super().__init__()

        self.input_layer = nn.Linear(input_size, hidden_size)
        self.bn_input = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()

        self.blocks = nn.ModuleList([
            ResidualBlock(hidden_size, dropout)
            for _ in range(num_blocks)
        ])

        self.fc_reduce = nn.Linear(hidden_size, hidden_size // 2)
        self.bn_reduce = nn.BatchNorm1d(hidden_size // 2)

        self.output_layer = nn.Linear(hidden_size // 2, 1)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                torch.nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.bn_input(x)
        x = self.relu(x)

        for block in self.blocks:
            x = block(x)

        x = self.fc_reduce(x)
        x = self.bn_reduce(x)
        x = self.relu(x)

        x = self.output_layer(x)
        return x

class VanillaANN(nn.Module):
    """Standard feedforward network"""
    def __init__(self, input_size, hidden_layers, dropout_rates, use_batchnorm=True):
        super().__init__()
        layers = []
        prev_size = input_size

        for hidden_size, dropout in zip(hidden_layers, dropout_rates):
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_size) if use_batchnorm else nn.Identity(),
                nn.Dropout(dropout) if dropout > 0 else nn.Identity()
            ])
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

### architecture configurations

In [6]:
VANILLA_CONFIGS = {
    'simple': {
        'hidden_layers': [256, 128],
        'dropout_rates': [0.3, 0.2],
        'learning_rate': 0.001,
        'batch_size': 256,
        'patience': 15
    },
    'moderate': {
        'hidden_layers': [256, 128, 64],
        'dropout_rates': [0.3, 0.3, 0.2],
        'learning_rate': 0.001,
        'batch_size': 256,
        'patience': 15
    }
}

RESNET_CONFIGS = {
    'light': {
        'hidden_size': 256,
        'num_blocks': 3,
        'dropout': 0.4,
        'learning_rate': 0.0003,
        'batch_size': 128,
        'patience': 25
    },
    'standard': {
        'hidden_size': 256,
        'num_blocks': 4,
        'dropout': 0.3,
        'learning_rate': 0.0003,
        'batch_size': 256,
        'patience': 25
    },
    'deep': {
        'hidden_size': 512,
        'num_blocks': 6,
        'dropout': 0.4,
        'learning_rate': 0.0001,
        'batch_size': 128,
        'patience': 30
    }
}

### Data fetching

In [7]:
def safe_float_conversion(value, property_name="unknown"):
    """
    Safely convert various types to float
    CRITICAL: Materials Project API returns elasticity values ALREADY in GPa
    Do NOT apply unit conversion - it destroys the data!
    """
    if value is None:
        return None

    try:
        # Handle emmet elasticity objects
        if hasattr(value, 'vrh'):
            val = float(value.vrh)  # Already in GPa from API

            # Sanity check: Filter physically impossible values
            if 'bulk_modulus' in property_name.lower():
                if val < 0 or val > 600:  # Bulk modulus: 0-600 GPa is physical range
                    return None
            elif 'shear_modulus' in property_name.lower():
                if val < 0 or val > 400:  # Shear modulus: 0-400 GPa is physical range
                    return None

            return val if not (np.isnan(val) or np.isinf(val)) else None

        elif hasattr(value, 'voigt'):
            val = float(value.voigt)  # Already in GPa

            if 'bulk_modulus' in property_name.lower():
                if val < 0 or val > 600:
                    return None
            elif 'shear_modulus' in property_name.lower():
                if val < 0 or val > 400:
                    return None

            return val if not (np.isnan(val) or np.isinf(val)) else None

        elif hasattr(value, 'reuss'):
            val = float(value.reuss)  # Already in GPa

            if 'bulk_modulus' in property_name.lower():
                if val < 0 or val > 600:
                    return None
            elif 'shear_modulus' in property_name.lower():
                if val < 0 or val > 400:
                    return None

            return val if not (np.isnan(val) or np.isinf(val)) else None

        # Handle Quantity objects
        elif hasattr(value, 'magnitude'):
            val = float(value.magnitude)
            return val if not (np.isnan(val) or np.isinf(val)) else None

        # Handle arrays
        elif hasattr(value, '__iter__') and not isinstance(value, str):
            arr = np.array(value)
            if arr.size > 0:
                val = float(np.mean(arr))
                return val if not (np.isnan(val) or np.isinf(val)) else None
            else:
                return None

        # Simple numeric
        else:
            val = float(value)
            return val if not (np.isnan(val) or np.isinf(val)) else None

    except Exception as e:
        return None

def remove_outliers(df, column, n_std=3, physical_bounds=None):
    """Remove outliers from a column"""
    if column not in df.columns:
        return df

    data = df[column].dropna()
    if len(data) < 50:
        return df

    mean = data.mean()
    std = data.std()

    lower_bound = mean - n_std * std
    upper_bound = mean + n_std * std

    if physical_bounds:
        lower_bound = max(lower_bound, physical_bounds[0])
        upper_bound = min(upper_bound, physical_bounds[1])

    mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)
    removed = (~mask & df[column].notna()).sum()

    if removed > 0:
        print(f"    ✂️ Removed {removed} outliers from {column}")

    df.loc[~mask, column] = np.nan
    return df

def fetch_by_query(mpr, query_params, max_results, description):
    """Generic fetch function"""
    print(f"  {description}...", end=" ")
    try:
        docs = mpr.materials.summary.search(
            **query_params,
            fields=["material_id", "formula_pretty", "nsites", "band_gap",
                   "formation_energy_per_atom", "energy_above_hull", "density", "volume"],
            chunk_size=500,
            num_chunks=min(20, (max_results // 500) + 1)
        )

        docs_list = [doc for i, doc in enumerate(docs) if i < max_results]
        print(f"✅ {len(docs_list)}")
        return docs_list
    except Exception as e:
        print(f"⚠️ Failed")
        return []

def fetch_diverse_materials(api_key, target_samples):
    """Fetch materials with all strategies"""

    print("\n" + "="*80)
    print("DIVERSE MATERIAL FETCHING")
    print("="*80)

    all_docs = []

    with MPRester(api_key) as mpr:

        print("\n📋 STRATEGY 1: By Element Count")
        print("-"*80)
        for n_elem in [1, 2, 3, 4, 5, 6]:
            docs = fetch_by_query(mpr, {'num_elements': (n_elem, n_elem)}, 5000, f"{n_elem}-elem")
            all_docs.extend(docs)
            time.sleep(1)

        print("\n📋 STRATEGY 2: By Stability")
        print("-"*80)
        for e_min, e_max in [(0, 0.001), (0.001, 0.01), (0.01, 0.05), (0.05, 0.1), (0.1, 0.2), (0.2, 0.5)]:
            docs = fetch_by_query(mpr, {'energy_above_hull': (e_min, e_max)}, 3000, f"hull {e_min}-{e_max}")
            all_docs.extend(docs)
            time.sleep(1)

        print("\n📋 STRATEGY 3: By Band Gap")
        print("-"*80)
        for bg_min, bg_max in [(0, 0.1), (0.1, 0.5), (0.5, 1.0), (1.0, 2.0), (2.0, 3.0), (3.0, 4.0), (4.0, 6.0), (6.0, 10.0)]:
            docs = fetch_by_query(mpr, {'band_gap': (bg_min, bg_max)}, 2500, f"BG {bg_min}-{bg_max}")
            all_docs.extend(docs)
            time.sleep(1)

        print("\n📋 STRATEGY 4: By Density")
        print("-"*80)
        for d_min, d_max in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 12), (12, 20)]:
            docs = fetch_by_query(mpr, {'density': (d_min, d_max)}, 2000, f"Density {d_min}-{d_max}")
            all_docs.extend(docs)
            time.sleep(1)

        print("\n📋 STRATEGY 5: By Formation Energy")
        print("-"*80)
        for fe_min, fe_max in [(-5, -3), (-3, -2), (-2, -1), (-1, 0), (0, 1)]:
            docs = fetch_by_query(mpr, {'formation_energy_per_atom': (fe_min, fe_max)}, 2000, f"FE {fe_min}-{fe_max}")
            all_docs.extend(docs)
            time.sleep(1)

        print(f"\n{'='*80}")
        print(f"TOTAL: {len(all_docs)} (before dedup)")
        print('='*80)

        # Process
        data = []
        for doc in all_docs:
            try:
                data.append({
                    "material_id": doc.material_id,
                    "formula": doc.formula_pretty,
                    "band_gap": getattr(doc, 'band_gap', None),
                    "formation_energy": getattr(doc, 'formation_energy_per_atom', None),
                    "energy_above_hull": getattr(doc, 'energy_above_hull', None),
                    "density": getattr(doc, 'density', None),
                    "volume_per_atom": (doc.volume / doc.nsites if hasattr(doc, 'volume') and doc.nsites > 0 else None)
                })
            except:
                continue

        df = pd.DataFrame(data)
        df = df.drop_duplicates(subset=['material_id'], keep='first')
        print(f"After dedup: {len(df)} {'✅' if len(df) >= target_samples else '⚠️'}")

        # Fetch additional properties
        print(f"\n{'='*80}")
        print("FETCHING ADDITIONAL PROPERTIES")
        print('='*80)

        material_ids = df['material_id'].tolist()
        batch_size = 5000

        all_mag, all_el, all_di, all_pz = [], [], [], []

        for i in range(0, len(material_ids), batch_size):
            batch_ids = material_ids[i:i+batch_size]
            print(f"\nBatch {i//batch_size + 1}: {len(batch_ids)} materials")

            # Magnetism
            try:
                print("  Magnetism...", end=" ")
                mag_docs = mpr.magnetism.search(material_ids=batch_ids,
                                               fields=["material_id", "total_magnetization"])
                mag_data = [{"material_id": d.material_id,
                            "total_magnetization": getattr(d, 'total_magnetization', None)}
                           for d in list(mag_docs)]
                all_mag.extend(mag_data)
                print(f"✅ {len(mag_data)}")
                time.sleep(0.5)
            except:
                print("⚠️")

            # Elasticity with FIXED unit handling
            try:
                print("  Elasticity...", end=" ")
                el_docs = mpr.elasticity.search(material_ids=batch_ids,
                                               fields=["material_id", "bulk_modulus", "shear_modulus",
                                                      "universal_anisotropy", "homogeneous_poisson"])

                el_docs_list = list(el_docs)

                el_data = []
                for d in el_docs_list:
                    try:
                        bulk = safe_float_conversion(getattr(d, 'bulk_modulus', None), 'bulk_modulus')
                        shear = safe_float_conversion(getattr(d, 'shear_modulus', None), 'shear_modulus')
                        aniso = safe_float_conversion(getattr(d, 'universal_anisotropy', None), 'anisotropy')
                        poisson = safe_float_conversion(getattr(d, 'homogeneous_poisson', None), 'poisson')

                        if any([bulk is not None, shear is not None, aniso is not None, poisson is not None]):
                            el_data.append({
                                "material_id": d.material_id,
                                "bulk_modulus": bulk,
                                "shear_modulus": shear,
                                "elastic_anisotropy": aniso,
                                "poissons_ratio": poisson
                            })
                    except:
                        continue

                all_el.extend(el_data)
                print(f"✅ {len(el_data)}")
                time.sleep(0.5)
            except:
                print("⚠️")

            # Dielectric
            try:
                print("  Dielectric...", end=" ")
                di_docs = mpr.dielectric.search(material_ids=batch_ids,
                                               fields=["material_id", "total", "electronic", "ionic"])
                di_data = []
                for d in list(di_docs):
                    try:
                        total = safe_float_conversion(getattr(d, 'total', None), 'dielectric_total')
                        electronic = safe_float_conversion(getattr(d, 'electronic', None), 'dielectric_electronic')
                        ionic = safe_float_conversion(getattr(d, 'ionic', None), 'dielectric_ionic')

                        di_data.append({
                            "material_id": d.material_id,
                            "dielectric_total": total,
                            "dielectric_electronic": electronic,
                            "dielectric_ionic": ionic
                        })
                    except:
                        continue
                all_di.extend(di_data)
                print(f"✅ {len(di_data)}")
                time.sleep(0.5)
            except:
                print("⚠️")

            # Piezoelectric
            try:
                print("  Piezoelectric...", end=" ")
                pz_docs = mpr.piezoelectric.search(material_ids=batch_ids,
                                                  fields=["material_id", "e_ij_max"])
                pz_data = [{"material_id": d.material_id,
                           "piezoelectric_max": getattr(d, 'e_ij_max', None)}
                          for d in list(pz_docs)]
                all_pz.extend(pz_data)
                print(f"✅ {len(pz_data)}")
            except:
                print("⚠️")

            time.sleep(1)

        # Merge
        if all_mag:
            df = df.merge(pd.DataFrame(all_mag), on='material_id', how='left')
        if all_el:
            df = df.merge(pd.DataFrame(all_el), on='material_id', how='left')
        if all_di:
            df = df.merge(pd.DataFrame(all_di), on='material_id', how='left')
        if all_pz:
            df = df.merge(pd.DataFrame(all_pz), on='material_id', how='left')

    # Remove outliers with physical bounds
    print(f"\n{'='*80}")
    print("REMOVING OUTLIERS (with physical bounds)")
    print('='*80)

    # Dielectric properties
    df = remove_outliers(df, 'dielectric_total', n_std=3, physical_bounds=(1.0, 100))
    df = remove_outliers(df, 'dielectric_electronic', n_std=3, physical_bounds=(1.0, 50))
    df = remove_outliers(df, 'dielectric_ionic', n_std=3, physical_bounds=(0, 100))

    # Piezoelectric
    df = remove_outliers(df, 'piezoelectric_max', n_std=3, physical_bounds=(0, 10))

    # Elasticity properties - CRITICAL: These have extreme outliers that break training
    df = remove_outliers(df, 'bulk_modulus', n_std=3, physical_bounds=(1, 500))
    df = remove_outliers(df, 'shear_modulus', n_std=3, physical_bounds=(1, 300))
    df = remove_outliers(df, 'poissons_ratio', n_std=3, physical_bounds=(0, 0.5))
    df = remove_outliers(df, 'elastic_anisotropy', n_std=3, physical_bounds=(0, 20))

    return df


### Training functions

In [8]:
def extract_features(df_prop, prop_name, elem_prop, stoich, valence):
    X_list, y_list = [], []
    for _, row in tqdm(df_prop.iterrows(), total=len(df_prop), desc="Featurizing"):
        try:
            comp = Composition(row['formula'])
            features = elem_prop.featurize(comp) + stoich.featurize(comp) + valence.featurize(comp)
            if not any(np.isnan(features)) and not any(np.isinf(features)):
                X_list.append(features)
                y_list.append(row[prop_name])
        except:
            continue
    return (np.vstack(X_list), np.array(y_list)) if X_list else (None, None)

def train_model(prop_name, prop_info, df, elem_prop, stoich, valence):
    if prop_name not in df.columns:
        return None

    df_prop = df.dropna(subset=[prop_name])
    if len(df_prop) < 200:
        print(f"\n⚠️ {prop_name}: Only {len(df_prop)} samples")
        return None

    print(f"\n{'='*80}\nTRAINING: {prop_info['description'].upper()}\n{'='*80}")

    X, y = extract_features(df_prop, prop_name, elem_prop, stoich, valence)
    if X is None or len(X) < 200:
        return None

    print(f"\n✅ {len(y)} samples")

    # CRITICAL: Check data quality before training
    y_mean = np.mean(y)
    y_std = np.std(y)
    y_min = np.min(y)
    y_max = np.max(y)

    print(f"   Data range: [{y_min:.4f}, {y_max:.4f}], mean={y_mean:.4f}, std={y_std:.4f}")

    # Sanity check: Detect if data is corrupted (all near zero)
    if y_std < 1e-6 or abs(y_mean) < 1e-6:
        print(f"⚠️  WARNING: Data variance too small (std={y_std:.6f}, mean={y_mean:.6f})")
        print(f"   This suggests data corruption - skipping training")
        return None

    # Sanity check: Detect extreme outliers that will break training
    if y_max > 1000 * y_mean or y_max > 10000:
        print(f"⚠️  WARNING: Extreme outliers detected (max={y_max:.1f}, mean={y_mean:.1f})")
        print(f"   Consider more aggressive outlier filtering")

    # Determine architecture
    use_resnet = prop_info.get('architecture') == 'resnet'

    if use_resnet:
        num_blocks = prop_info.get('resnet_blocks', 4)
        if len(y) < 2000:
            config_name = 'light'
        elif num_blocks >= 6:
            config_name = 'deep'
        else:
            config_name = 'standard'

        config = RESNET_CONFIGS[config_name]
        print(f"🔧 Architecture: ResNet-{num_blocks} ({config_name}) - {config['hidden_size']} hidden")
    else:
        config_name = 'simple' if prop_name in ['formation_energy', 'density'] else 'moderate'
        config = VANILLA_CONFIGS[config_name]
        print(f"🔧 Architecture: Vanilla ({config_name}) - {config['hidden_layers']}")

    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Scale
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s, X_test_s = scaler.transform(X_val), scaler.transform(X_test)

    # Create model
    if use_resnet:
        model = ResNetPredictor(
            X_train_s.shape[1],
            hidden_size=config['hidden_size'],
            num_blocks=num_blocks,
            dropout=config['dropout']
        ).to(DEVICE)
    else:
        hidden_layers = config['hidden_layers']
        dropout_rates = config['dropout_rates']
        if len(dropout_rates) < len(hidden_layers):
            dropout_rates = dropout_rates + [dropout_rates[-1]] * (len(hidden_layers) - len(dropout_rates))

        model = VanillaANN(
            X_train_s.shape[1],
            hidden_layers,
            dropout_rates,
            use_batchnorm=True
        ).to(DEVICE)

    # Optimizer with weight decay for regularization
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=0.01)

    # Cosine annealing with warm restarts - better than ReduceLROnPlateau for stability
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=20, T_mult=2, eta_min=config['learning_rate'] * 0.01
    )

    criterion = nn.MSELoss()

    # DataLoaders
    train_loader = DataLoader(
        TensorDataset(torch.FloatTensor(X_train_s), torch.FloatTensor(y_train).reshape(-1,1)),
        config['batch_size'], shuffle=True
    )
    val_loader = DataLoader(
        TensorDataset(torch.FloatTensor(X_val_s), torch.FloatTensor(y_val).reshape(-1,1)),
        config['batch_size']
    )

    print("🚀 Training...")
    best_loss = float('inf')
    patience_counter = 0
    patience = config['patience']

    for epoch in range(EPOCHS):
        # Train
        model.train()
        train_loss = 0
        for X_b, y_b in train_loader:
            X_b, y_b = X_b.to(DEVICE), y_b.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(X_b), y_b)
            loss.backward()

            # Gradient clipping for ResNet (prevents exploding gradients)
            if use_resnet:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_b, y_b in val_loader:
                val_loss += criterion(model(X_b.to(DEVICE)), y_b.to(DEVICE)).item()
        val_loss /= len(val_loader)

        # Update learning rate
        scheduler.step()

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), f'ann_{prop_name}.pt')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"   Early stop at epoch {epoch+1}")
                break

        if (epoch+1) % 20 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, LR = {current_lr:.6f}")

    # Test
    try:
        model.load_state_dict(torch.load(f'ann_{prop_name}.pt'))
        model.eval()
        with torch.no_grad():
            y_pred = model(torch.FloatTensor(X_test_s).to(DEVICE)).cpu().numpy().flatten()

        metrics = {
            'mae': mean_absolute_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'r2': r2_score(y_test, y_pred),
            'architecture': 'ResNet' if use_resnet else 'Vanilla',
            'samples': len(y)
        }

        torch.save({
            'model_state_dict': model.state_dict(),
            'input_size': X_train_s.shape[1],
            'config': config,
            'metrics': metrics,
            'architecture_type': 'resnet' if use_resnet else 'vanilla'
        }, f'ann_{prop_name}.pt')

        joblib.dump(scaler, f'scaler_{prop_name}.pkl')

        print(f"\n✅ MAE: {metrics['mae']:.4f}, R²: {metrics['r2']:.4f}")
        return metrics
    except Exception as e:
        print(f"❌ Error during evaluation: {str(e)}")
        return None


### Main execution

In [9]:
start_time = time.time()
df_master = fetch_diverse_materials(MP_API_KEY, TARGET_SAMPLES)
fetch_time = time.time() - start_time

print(f"\n✅ Fetch: {fetch_time/60:.1f} min, {len(df_master)} materials")

if len(df_master) < 1000:
    raise SystemExit("❌ Insufficient data")

# Statistics
print("\n" + "="*80)
print("DATASET STATISTICS")
print("="*80)
for prop_name, prop_info in PROPERTIES.items():
    if prop_name in df_master.columns:
        count = df_master[prop_name].notna().sum()
        arch = prop_info.get('architecture', 'vanilla')
        if count > 0:
            print(f"{prop_info['description']:40s} {count:6d} [{arch.upper()}]")

# Features
elem_prop = ElementProperty.from_preset("magpie")
stoich = Stoichiometry()
valence = ValenceOrbital()
feature_names = elem_prop.feature_labels() + stoich.feature_labels() + valence.feature_labels()
joblib.dump(feature_names, 'feature_names.pkl')
print(f"\n✅ {len(feature_names)} features")

# Train
print("\n" + "="*80)
print("TRAINING ALL MODELS - FINAL PRODUCTION RUN")
print("="*80)

results = {}
for prop_name, prop_info in PROPERTIES.items():
    try:
        metrics = train_model(prop_name, prop_info, df_master, elem_prop, stoich, valence)
        if metrics:
            results[prop_name] = metrics
    except Exception as e:
        print(f"❌ {prop_name}: {str(e)[:100]}")

# Summary
print("\n" + "="*80)
print("FINAL PRODUCTION RESULTS")
print("="*80)
print(f"Time: {(time.time() - start_time)/60:.1f} min")
print(f"Properties: {len(results)}/{len(PROPERTIES)}\n")

# Group by architecture
resnet_props = [p for p, info in PROPERTIES.items() if info.get('architecture') == 'resnet' and p in results]
vanilla_props = [p for p, info in PROPERTIES.items() if info.get('architecture') != 'resnet' and p in results]

if resnet_props:
    print("\n🧠 RESNET MODELS:")
    print("-"*80)
    for prop_name in resnet_props:
        metrics = results[prop_name]
        print(f"{PROPERTIES[prop_name]['description']:40s} MAE={metrics['mae']:8.4f}  R²={metrics['r2']:6.3f}")

if vanilla_props:
    print("\n⚡ VANILLA MODELS:")
    print("-"*80)
    for prop_name in vanilla_props:
        metrics = results[prop_name]
        print(f"{PROPERTIES[prop_name]['description']:40s} MAE={metrics['mae']:8.4f}  R²={metrics['r2']:6.3f}")

# Count by performance
excellent = sum(1 for m in results.values() if m['r2'] > 0.75)
good = sum(1 for m in results.values() if 0.50 <= m['r2'] <= 0.75)
moderate = sum(1 for m in results.values() if 0.30 <= m['r2'] < 0.50)
poor = sum(1 for m in results.values() if 0.10 <= m['r2'] < 0.30)
failed = sum(1 for m in results.values() if m['r2'] < 0.10)

print(f"\n📊 PERFORMANCE BREAKDOWN:")
print(f"   🏆 Excellent (R² > 0.75): {excellent}")
print(f"   ✅ Good (R² 0.50-0.75): {good}")
print(f"   ⚠️  Moderate (R² 0.30-0.50): {moderate}")
print(f"   ⚠️  Poor (R² 0.10-0.30): {poor}")
print(f"   ❌ Failed (R² < 0.10): {failed}")
print(f"\n   🎯 Production-Ready (R² > 0.30): {excellent + good + moderate}/{len(PROPERTIES)}")
success_rate = (excellent + good + moderate) / len(PROPERTIES) * 100
print(f"   📈 Success Rate: {success_rate:.1f}%")

print("\n✅ Completed!")
print("="*80)



DIVERSE MATERIAL FETCHING

📋 STRATEGY 1: By Element Count
--------------------------------------------------------------------------------
  1-elem... 

Retrieving SummaryDoc documents:   0%|          | 0/834 [00:00<?, ?it/s]

✅ 834
  2-elem... 

Retrieving SummaryDoc documents:   0%|          | 0/5500 [00:00<?, ?it/s]

✅ 5000
  3-elem... 

...


✅ 1354 samples
   Data range: [1.0005, 41.9319], mean=2.6589, std=3.0479
🔧 Architecture: ResNet-6 (light) - 256 hidden
🚀 Training...
   Epoch 20: Train Loss = 25.9293, Val Loss = 22.6002, LR = 0.000300
   Epoch 40: Train Loss = 7.3461, Val Loss = 19.8551, LR = 0.000151
   Epoch 60: Train Loss = 6.6925, Val Loss = 19.8726, LR = 0.000300
   Early stop at epoch 64

✅ MAE: 1.1150, R²: 0.5062

TRAINING: DIELECTRIC (IONIC)


Featurizing: 100%|██████████| 1992/1992 [00:05<00:00, 336.04it/s]



✅ 1992 samples
   Data range: [0.0000, 82.8646], mean=3.6492, std=5.8758
🔧 Architecture: Vanilla (moderate) - [256, 128, 64]
🚀 Training...
   Epoch 20: Train Loss = 27.1319, Val Loss = 34.3811, LR = 0.001000
   Epoch 40: Train Loss = 18.4908, Val Loss = 25.0502, LR = 0.000505
   Epoch 60: Train Loss = 14.5685, Val Loss = 18.9144, LR = 0.001000
   Epoch 80: Train Loss = 12.2207, Val Loss = 17.1662, LR = 0.000855
   Epoch 100: Train Loss = 10.4611, Val Loss = 12.1163, LR = 0.000505
   Early stop at epoch 117

✅ MAE: 2.6213, R²: 0.1210

TRAINING: PIEZOELECTRIC MAX


Featurizing: 100%|██████████| 859/859 [00:02<00:00, 334.55it/s]



✅ 859 samples
   Data range: [0.0000, 7.6901], mean=0.6134, std=0.8393
🔧 Architecture: Vanilla (moderate) - [256, 128, 64]
🚀 Training...
   Epoch 20: Train Loss = 0.8699, Val Loss = 0.8150, LR = 0.001000
   Epoch 40: Train Loss = 0.5039, Val Loss = 0.6779, LR = 0.000505
   Epoch 60: Train Loss = 0.4440, Val Loss = 0.6732, LR = 0.001000
   Epoch 80: Train Loss = 0.3494, Val Loss = 0.6238, LR = 0.000855
   Early stop at epoch 88

✅ MAE: 0.5276, R²: -0.2450

FINAL PRODUCTION RESULTS
Time: 22.9 min
Properties: 14/14


🧠 RESNET MODELS:
--------------------------------------------------------------------------------
Band Gap                                 MAE=  0.5067  R²= 0.769
Magnetization                            MAE=  2.6660  R²= 0.580
Dielectric (Total)                       MAE=  4.1534  R²=-0.078
Dielectric (Electronic)                  MAE=  1.1150  R²= 0.506

⚡ VANILLA MODELS:
--------------------------------------------------------------------------------
Formation Energy     